In [252]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions
import time
from selenium.webdriver.common.keys import Keys

In [253]:
driverpath="C:/Users/mohit.chaniyal/Downloads/chromedriver.exe" #provide driver location here

In [254]:
# with open("myinstacred.txt","r") as f:
#     uname,passwd=[x.replace("\n","") for x in f.readlines()]
uname="Your Username Here"  
passwd="Your Password Here"

In [255]:
mobile_emulation = {"deviceName": "iPhone X"}
chrome_options = ChromeOptions()
chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)
driver =Chrome(options=chrome_options,executable_path=driverpath)
driver.get("https://www.instagram.com/")

driver.set_window_size(375, 812)  # iPhone X resolution: 375x812

C:\Users\mohit.chaniyal\AppData\Local\Temp\ipykernel_60736\2579061682.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver =Chrome(options=chrome_options,executable_path=driverpath)


In [256]:
#login
time.sleep(2)
driver.find_element(By.XPATH,"//div[text()='Log in']").click()
time.sleep(2)
driver.find_element(By.NAME,"username").send_keys(uname)
driver.find_element(By.NAME,"password").send_keys(passwd)
driver.find_element(By.XPATH,"//div[text()='Log in']").click()


In [200]:
#scraping all the instagram followers

driver.get(f"https://www.instagram.com/{uname}/followers/")
time.sleep(3)
prev_height=driver.execute_script("return document.body.scrollHeight")
followers_set=set()
while True :
    followers_list=driver.find_elements(By.XPATH,"//span[@class='_aacl _aaco _aacw _aacx _aad7 _aade']/div")
    for follower in followers_list :
        if follower.text not in followers_set:
            followers_set.add(follower.text)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)
    new_height=driver.execute_script("return document.body.scrollHeight")
    if new_height==prev_height:
        break
    prev_height=new_height

In [150]:
#scraping all the the account i follow
driver.get(f"https://www.instagram.com/{uname}/following/") 
time.sleep(3)
prev_height=driver.execute_script("return document.body.scrollHeight")
followings_set=set()
while True :
    followings_list=driver.find_elements(By.XPATH,"//span[@class='_aacl _aaco _aacw _aacx _aad7 _aade']/div")
    for following in followings_list :
        if following.text not in followings_set:
            followings_set.add(following.text)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)
    new_height=driver.execute_script("return document.body.scrollHeight")
    if new_height==prev_height:
        break
    prev_height=new_height

In [217]:
#saving data to csv files
import pandas as pd
pd.DataFrame(followings_set,columns=["Following"])["Following"].apply(lambda x : x.replace("\nVerified","")).to_csv("following.csv",index=False)
pd.DataFrame(followers_set,columns=["Followers"])["Followers"].apply(lambda x : x.replace("\nVerified","")).to_csv("followers.csv",index=False)

In [218]:
df1=pd.read_csv("following.csv")

In [220]:
df2=pd.read_csv("followers.csv")

In [237]:
person_i_follow_but_they_do_not_follow_me=df1[~df1.Following.isin(df2.Followers)]

In [239]:
pd.set_option('display.max_rows', 200)

In [ ]:
person_i_follow_but_they_do_not_follow_me

In [ ]:
#creating another instance becaouse facing issue with the display size while unfollowing
browser=Chrome(executable_path=driverpath)
browser.get("https://www.instagram.com/")
#login
time.sleep(2)
browser.find_element(By.NAME,"username").send_keys(uname)
browser.find_element(By.NAME,"password").send_keys(passwd)
browser.find_element(By.XPATH,"//div[text()='Log in']").click()

In [ ]:
#unfollowing people who dont follow me and are not verified
for person in person_i_follow_but_they_do_not_follow_me.Following:
    browser.get(f"https://www.instagram.com/{person}")
    time.sleep(5)
    try:
        browser.find_element(By.XPATH,"//span[text()='Verified']")
        continue
    except:
        try :
            browser.find_element(By.XPATH,"//div[text()='Following']").click()
            time.sleep(5)
            browser.find_element(By.XPATH,"//span[text()='Unfollow']").click()
            time.sleep(2)
        except:
            print(pe)